# Hyperopt (Logit Leaf Model)

With the use of this notebook, the optimal parameters for the Logit Leaf Model (LLM) are determined
Note that this code is based on the code provided by https://github.com/irhete/predictive-monitoring-benchmark

#### dataset BPIC2015 2 f2

To find out how these columns where determined, see the Feature Selection file.

In [32]:
#12 columns (20% correlation)

In [33]:
columns_PC20= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other']

In [34]:
#22 columns (15% correlation)

In [35]:
columns_PC15= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other',
 'agg__Activity_08_AWB45_060',
 'agg__Activity_01_HOOFD_191',
 'agg__Activity_08_AWB45_051_0',
 'agg__timesincelastevent_sum',
 'agg__Activity_01_HOOFD_490_1',
 'agg__Activity_01_HOOFD_250',
 'agg__Activity_08_AWB45_090_2',
 'static__Kap',
 'static__Milieu (neutraal wijziging)',
 'agg__Activity_01_HOOFD_490_2']

In [36]:
#50 columns (10% correlation)

In [37]:
columns_PC10= ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other',
 'agg__Activity_08_AWB45_060',
 'agg__Activity_01_HOOFD_191',
 'agg__Activity_08_AWB45_051_0',
 'agg__timesincelastevent_sum',
 'agg__Activity_01_HOOFD_490_1',
 'agg__Activity_01_HOOFD_250',
 'agg__Activity_08_AWB45_090_2',
 'static__Kap',
 'static__Milieu (neutraal wijziging)',
 'agg__Activity_08_AWB45_070_3',
 'static__Gebiedsbescherming',
 'agg__month_std',
 'agg__timesincecasestart_std',
 'agg__Activity_01_HOOFD_250_1',
 'agg__question_42',
 'agg__Activity_01_HOOFD_130',
 'agg__Activity_01_HOOFD_250_2',
 'agg__weekday_std',
 'agg__Activity_08_OLO_100',
 'agg__Activity_01_HOOFD_190_1',
 'static__Responsible_actor_560530',
 'agg__Activity_11_AH_II_010',
 'agg__Activity_08_AWB45_050',
 'agg__timesincecasestart_mean',
 'agg__weekday_max',
 'agg__timesincelastevent_max',
 'agg__Activity_08_AWB45_090_1',
 'agg__timesincelastevent_std',
 'agg__Activity_08_AWB45_020_0',
 'agg__timesincecasestart_sum',
 'agg__open_cases_std',
 'agg__timesincelastevent_mean',
 'agg__Activity_01_HOOFD_190_2',
 'static__Inrit/Uitweg',
 'agg__Activity_01_HOOFD_470',
 'agg__Activity_01_HOOFD_510_2',
 'agg__month_min',
 'agg__Activity_01_HOOFD_495']

#### traffic_fines1

In [23]:
#15% PC

In [24]:
columns_PC15 = ['agg__Activity_Payment', 'agg__Activity_Send Fine']

In [25]:
#10% PC

In [26]:
columns_PC10 = ['agg__Activity_Payment', 'agg__Activity_Send Fine', 'agg__Activity_Insert Date Appeal to Prefecture']

#### bpic2017_accepted

In [27]:
#20% PC

In [8]:
columns_PC20= ['agg__CreditScore_std', 'agg__Selected_False']

In [9]:
#15% PC

In [10]:
columns_PC15= ['agg__CreditScore_std',
 'agg__Selected_False',
 'static__ApplicationType_New credit',
 'static__ApplicationType_Limit raise']

In [11]:
#10% PC

In [12]:
columns_PC10 = ['static__ApplicationType_Limit raise',
 'static__ApplicationType_New credit',
 'agg__Activity_A_Cancelled',
 'agg__Activity_A_Submitted',
 'agg__Activity_W_Validate application',
 'agg__lifecycle:transition_ate_abort',
 'agg__Accepted_False',
 'agg__Accepted_True',
 'agg__Selected_False',
 'agg__CreditScore_std']

### run

In [41]:
import pandas as pd
import numpy as np

import time
import os
import random

import EncoderFactory
from DatasetManager import DatasetManager

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from tabulate import tabulate

import pickle

import hyperopt
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll.base import scope
from aix360.algorithms.rbm import LogisticRuleRegression
from aix360.algorithms.rbm import FeatureBinarizer

import xgboost as xgb


def create_and_evaluate_model(args): 
    global trial_nr
    trial_nr += 1
    
    start = time.time()
    score = 0
    for cv_iter in range(n_splits):
        
        dt_test_prefixes = dt_prefixes[cv_iter]
        dt_train_prefixes = pd.DataFrame()
        for cv_train_iter in range(n_splits): 
            if cv_train_iter != cv_iter:
                dt_train_prefixes = pd.concat([dt_train_prefixes, dt_prefixes[cv_train_iter]], axis=0)
        
        #remove training rows with negative values (should be a mistake)
        #dt_train_prefixes = dt_train_prefixes[dt_train_prefixes.select_dtypes(include=[np.number]).ge(0).all(1)]

        auc_totals = []
        auc_best = 0

        preds_all = []
        test_y_all = []
        test_y = dataset_manager.get_label_numeric(dt_test_prefixes)
        train_y = dataset_manager.get_label_numeric(dt_train_prefixes)  
        test_y_all.extend(test_y)       
        #feature combiner
        feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])
        feature_combiner.fit(dt_train_prefixes, train_y)

        #transform train dataset
        dt_train_named= feature_combiner.transform(dt_train_prefixes)
        dt_train_named = pd.DataFrame(dt_train_named)
        names= feature_combiner.get_feature_names()
        dt_train_named.columns = names
        dt_train_named = dt_train_named[columns] 
        
        #transform test dataset
        dt_test_named = feature_combiner.transform(dt_test_prefixes)
        dt_test_named = pd.DataFrame(dt_test_named)
        names= feature_combiner.get_feature_names()
        dt_test_named.columns = names
        dt_test_named = dt_test_named[columns]
        if cls_method =='xgboost':
            cls = xgb.XGBClassifier(objective='binary:logistic',
                                                n_estimators=500,
                                                learning_rate= args['learning_rate'],
                                                subsample=args['subsample'],
                                                max_depth=int(args['max_depth']),
                                                colsample_bytree=args['colsample_bytree'],
                                                min_child_weight=int(args['min_child_weight']),
                                                seed=random_state)

            cls.fit(dt_train_named, train_y)
            preds_pos_label_idx = np.where(cls.classes_ == 1)[0][0]
            pred = cls.predict_proba(dt_test_named)[:,preds_pos_label_idx]
            preds_all.extend(pred)
            
        elif cls_method =='glrm':
            fb = FeatureBinarizer(negations=False, returnOrd=True, threshStr=False, numThresh=10 ) 
            dt_train_named, dfTrainStd = fb.fit_transform(dt_train_named)
            dt_test_named, dfTestStd = fb.transform(dt_test_named)
            cls = LogisticRuleRegression(lambda0=args['lambda0'], lambda1=args['lambda1'],
                                         iterMax=10000)
            # Train, print, and evaluate model
            cls.fit(dt_train_named, pd.Series(train_y))
            pred = cls.predict_proba(dt_test_named)
            preds_all.extend(pred)
        auc_total = roc_auc_score(test_y_all, preds_all)
        score += roc_auc_score(test_y_all, preds_all)
    
    
       
        for k, v in args.items():
            fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, k, v, score / n_splits))   
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, "processing_time", time.time() - start, 0))   
    fout_all.flush()
    return {'loss': -score / n_splits, 'status': STATUS_OK, 'model': cls}

dataset_ref = 'bpic2015_2_f2'
params_dir = './params_dir'
n_iter = 5
cls_encoding = 'agg'
cls_method = 'xgboost'

column_selection = 'PC20'
columns = columns_PC20

method_name = "%s_%s"%(column_selection, cls_encoding)

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
    "insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"],
    "bpic2015_half": ["bpic2015_%s_f2_half"%(municipality) for municipality in range(1,6)]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"],
    "onehot": ["static","onehot"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
    
train_ratio = 0.8
n_splits = 3
random_state = 22

# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))
    
for dataset_name in datasets:
    
    # read the data
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

    # split into training and test
    train, _ = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    
    
    # prepare chunks for CV
    dt_prefixes = []
    class_ratios = []
    for train_chunk, test_chunk in dataset_manager.get_stratified_split_generator(train, n_splits=n_splits):
        class_ratios.append(dataset_manager.get_class_ratio(train_chunk))
        # generate data where each prefix is a separate instance
        dt_prefixes.append(dataset_manager.generate_prefix_data(test_chunk, min_prefix_length, max_prefix_length))
    del train
        
    # set up search space
    
    if cls_method == "xgboost":
        space = {'learning_rate': hp.uniform("learning_rate", 0, 1),
                 'subsample': hp.uniform("subsample", 0.5, 1),
                 'max_depth': scope.int(hp.quniform('max_depth', 4, 30, 1)),
                 'colsample_bytree': hp.uniform("colsample_bytree", 0.5, 1),
                 'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 6, 1))}
      
    if cls_method =='glrm':
        space = {'lambda0': hp.uniform("lambda0",0.00001, 0.01),
                 'lambda1': hp.uniform("lambda1", 0.00001,0.01)}
    
    
    # optimize parameters
    trial_nr = 0
    trials = Trials()
    fout_all = open(os.path.join(params_dir, "param_optim_all_trials_%s_%s_%s.csv" % (cls_method, dataset_name, method_name)), "w")
    if "prefix" in method_name:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "nr_events", "param", "value", "score"))   
    else:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "param", "value", "score"))   
    rstate = np.random.RandomState(22)
    best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals=n_iter, trials=trials, rstate = rstate)
    fout_all.close()

    # write the best parameters
    best_params = hyperopt.space_eval(space, best)
    outfile = os.path.join(params_dir, "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))
    # write to file
    with open(outfile, "wb") as fout:
        pickle.dump(best_params, fout)

100%|█████████████████████████████████████████████████| 5/5 [02:50<00:00, 34.16s/trial, best loss: -0.9229344126157142]
